In [17]:
# Dependencies
import pandas as pd
import sys
import numpy as np
import requests

# Access config file
sys.path.append('../')

# API stuff
import gmaps
from citipy import citipy
from config import weather_api_key as w_key
from config import g_key

In [11]:
# Generate random lats and lngs then convert to 2-D array
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-180, 180, size=2000)
coords = zip(lats, lngs)
coords = list(coords)

In [29]:
# Get city names for city nearest to each set of coords
cities = [citipy.nearest_city(coord[0], coord[1]).city_name for coord in coords]
cities = pd.unique(cities)
len(cities)

735

In [39]:
# Call OpenWeatherMap API for each city to gather current weather data
url_base = 'http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=' + w_key + '&q='

# Initialize list to store each iteration of data
city_data = []

print('-------------------------')
print('----Starting API call----')
print('-------------------------')
index = 1
set_num = 1

# Need index and list items => use enumerate
for i, city in enumerate(cities):
    
    # Track API calls
    if i % 50 == 0 and i >= 50:
        index = 1
        set_num += 1
    print(f'iteration #{index} in set #{set_num} | {city}')

    index += 1

    # Gather data in try-except block in case some cities in list aren't found by API call
    try:
        city_weather = requests.get(url_base + city).json()
        # Store data in list of dictionaries
        city_data.append({
                    'City': city,
                    'Country': city_weather['sys']['country'],
                    'Lat': city_weather['coord']['lat'],
                    'Lng': city_weather['coord']['lon'],
                    'Max Temp': city_weather['main']['temp_max'],
                    'Humidity': city_weather['main']['humidity'],
                    'Cloudiness': city_weather['clouds']['all'],
                    'Wind Speed': city_weather['wind']['speed'],
                    'Current Description': city_weather['weather'][0]['description']
    })
    except:
        print(city + 'not found. Skipping...')
        continue

print('-------------------------')
print('----API Call Complete----')
print('-------------------------')

-------------------------
----Starting API call----
-------------------------
iteration #1 in set #1 | cape town
iteration #2 in set #1 | thompson
iteration #3 in set #1 | jaisalmer
iteration #4 in set #1 | nabire
iteration #5 in set #1 | new norfolk
iteration #6 in set #1 | nayoro
iteration #7 in set #1 | east london
iteration #8 in set #1 | qaanaaq
iteration #9 in set #1 | port hardy
iteration #10 in set #1 | raudeberg
iteration #11 in set #1 | mackay
iteration #12 in set #1 | jamestown
iteration #13 in set #1 | dubbo
iteration #14 in set #1 | yellowknife
iteration #15 in set #1 | avarua
iteration #16 in set #1 | kamakwie
iteration #17 in set #1 | rikitea
iteration #18 in set #1 | punta arenas
iteration #19 in set #1 | vostok
iteration #20 in set #1 | bredasdorp
iteration #21 in set #1 | busselton
iteration #22 in set #1 | vaini
iteration #23 in set #1 | carnarvon
iteration #24 in set #1 | hilo
iteration #25 in set #1 | hermanus
iteration #26 in set #1 | cherskiy
iteration #27 in set

In [41]:
# Add the data to a DataFrame
weather_df = pd.DataFrame(city_data)
weather_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,cape town,ZA,-33.9258,18.4232,58.87,88,75,3.00,broken clouds
1,thompson,CA,55.7435,-97.8558,53.76,54,90,13.80,overcast clouds
2,jaisalmer,IN,26.9147,70.9181,87.66,57,100,8.90,overcast clouds
3,nabire,ID,-3.3667,135.4833,74.30,92,72,3.22,light rain
4,new norfolk,AU,-42.7826,147.0587,42.93,79,11,1.99,few clouds
5,nayoro,JP,44.3506,142.4578,59.94,97,100,4.59,overcast clouds
6,east london,ZA,-33.0153,27.9116,80.29,39,5,6.55,clear sky
7,qaanaaq,GL,77.4840,-69.3632,22.53,67,10,7.47,clear sky
8,port hardy,CA,50.6996,-127.4199,60.98,77,75,4.61,light rain
9,raudeberg,NO,61.9875,5.1352,56.19,95,100,1.01,overcast clouds


In [43]:
file_to_save = 'WeatherPy_Database.csv'
weather_df.to_csv(file_to_save, index_label='City_ID')